# Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

from table_enforcer import Enforcer, Column
import table_enforcer.errors as e

from table_enforcer import validate as v
from table_enforcer import recode as r

# Load or create your Table

In [3]:
df = pd.read_csv("demo_table.csv")
df

,col1,col2,col3,col4
0,7,3,10,male
1,2,4,6,m
2,6,6,2,F


# Validator Functions

## Built-in Validators

Some validator functions come built-in (See the `table_enforcer/validate/funcs.py` module code for current list)

In [4]:
! cat $v.funcs.__file__

"""Provide builtin validator functions for common use cases.

In general, validators take a single `pandas.Series` object as
input and return a `pandas.Series` of the same shape and indexes
containing `True` or `False` relative to which items passed the
validation logic.
"""
import pandas as pd
# import numpy as np

# from table_enforcer import errors as e
# from table_enforcer.validate import decorators as dec


def not_null(series: pd.Series) -> pd.Series:
    """Return Series with True/False bools based on which items pass."""
    return pd.notnull(series)


def positive(series: pd.Series) -> pd.Series:
    """Test that the data items are positive."""
    return series > 0


def negative(series: pd.Series) -> pd.Series:
    """Test that the data items are negative."""
    return series < 0


def unique(series: pd.Series) -> pd.Series:
    """Test that the data items do not repeat."""
    return ~series.duplicated(keep=False)


def upper(series):
    """Test that the data items are a

## Validator Call Signatures

In general, validators take a single `pandas.Series` object as input and return a `pandas.Series` of the same shape and indexes containing `True` or `False` relative to which items passed the validation logic. 

This changes a little if you are using a decorator like `@maxmin` (_See section on Decorating Validators_)



## Writing Custom Validators

In [5]:
def gte2(series):
    return series >= 2

def lte10(series):
    return series <= 10

def length_is_one(series):
    return series.str.len() == 1

def valid_sex(series):
    sex = set(['M', 'F'])
    return series.isin(sex)

## Decorating Validators
You can define decorators that extend the call signature of validators to accomadate more complex things like ranges.

Here we can define a single validator that accomplishes the same thing as the combination of the first two validators that we defined above:

In [6]:
@v.decorators.minmax(low=2, high=10)
def bt_2_and_10(series):
    """Test that the data items fall within range: 2 <= x <= 10."""
    return series

__Note:__ pay attention to the fact that here we return the __original__ series object. _We didn't do ANYTHING to it._ The testing gets done in the `@v.decorators.minmax` decorator function. This allows our validators to always expect a single argument at run-time and always return a single argument.  This is _very_ important.

### Writing Custom Validator Decorators

This is a bit beyond this demo's scope but here is the `table_enforcer/validate/decorators.py` code so that you can get an idea oh how to do it if you are brave.

In [7]:
!cat $v.decorators.__file__

"""Provide decoration functions to augment the behavior of validator functions."""
import functools

import pandas as pd
import numpy as np


def minmax(low, high):
    """Test that the data items fall within range: low <= x <= high."""
    def decorator(function):
        """Decorate a function with args."""
        @functools.wraps(function)
        def wrapper(*args, **kwargs):
            """Wrap the function."""
            series = function(*args, **kwargs)
            lo_pass = low <= series
            hi_pass = series <= high

            return lo_pass & hi_pass

        return wrapper

    return decorator


def choice(choices):
    """Test that the data items are members of the set `choices`."""
    def decorator(function):
        """Decorate a function with args."""
        @functools.wraps(function)
        def wrapper(*args, **kwargs):
            """Wrap the function."""
            series = function(*args, **kwargs)
            return series.isin(set(choices))

      

# Defining Columns

Now that we have a few validator functions, its time to put together a representation of our table and use that object to help us get us a final data table that is in a form we like. We will start with `col1`.

In [8]:
df

,col1,col2,col3,col4
0,7,3,10,male
1,2,4,6,m
2,6,6,2,F


Lets say we don't care about `col2`; we want all the others though.

We create a column object for each column we want that contains the name of the column we want to represent. We set the options to describe our ideal format. We set the values for each option for each column including a a list of validator functions that should all pass for all items in each column if we got our way.

For now, we will ignore the `recoders` argument. What we end up with is the three column objects below.

In [9]:
col1 = Column(name='col1',
             dtype=np.int,
             unique=False,
             validators=[v.funcs.not_null, v.funcs.positive, bt_2_and_10],
             recoders=None)

col3 = Column(name='col3',
             dtype=np.int,
             unique=True,
             validators=[v.funcs.not_null, v.funcs.positive, bt_2_and_10],
             recoders=None)

col4 = Column(name='col4',
             dtype=str,
             unique=False,
             validators=[v.funcs.upper, length_is_one, valid_sex],
             recoders=None)

## Validating a Column object
For each column object we store the validator functions in a `pd.Series`.

Here is the value for `col4`

In [10]:
col4.validators

{'length_is_one': <function __main__.length_is_one>,
 'upper': <function table_enforcer.validate.funcs.upper>,
 'valid_sex': <function __main__.valid_sex>}

We validate a column by passing our original dataframe to the column object. This means that the name argument we pass to our column object __must match the target column header EXACTLY__. The tests get run independently and after the column data is passed through each test, we get a dataframe of results.  We can see right away which data items fail which tests.

This helps us plan our recoding efforts later. We can immediatly see what sorts of operations need to be run to convert those `False` tests to `True`.

In [11]:
col4.validate(df)

,length_is_one,upper,valid_sex,dtype
0,False,False,False,True
1,True,False,False,True
2,True,True,True,True


Wait a minute... Where did that validation test called `dtype` come from?

Turns out we get that one for free since we defined that this column needs to be of `dtype` `np.int`. We get a similar `unique` validation check inserted if we set that option to `True`. 

# Recoder Functions

Recoder functions have a very similar structure to validator functions. But their purpose is to coerce the raw data into a more useful form. For example `Col4` is obviously meant to represent 'male'/'female' logic. But we don't want to have to interpret all possible ways to represent that logic every time we want to use that column. So we write a recoder function to do convert all items to either "M" or "F" so we only need to understand two choices from now on.

In [12]:
df

,col1,col2,col3,col4
0,7,3,10,male
1,2,4,6,m
2,6,6,2,F


## Built-in Recoders

Some recoder functions come built-in (See the `table_enforcer/recode/funcs.py` module code for current list)

In [13]:
!cat $r.funcs.__file__

"""Provide builtin recoder functions for common use cases.

Like validators, recoders take a single `pandas.Series` object as input and return
a `pandas.Series` of the same shape and indexes as the original series object. However,
instead of returning a series of `True`/`False` values, it performs some operation on
the data that gets the column data closer to being how you want it to look during
analysis operations.
"""


def upper(series):
    """Transform all text to uppercase."""
    return series.str.upper()


def lower(series):
    """Transform all text to lowercase."""
    return series.str.lower()

## Recoder Call Signatures

Like validators, recoders take a single `pandas.Series` object as input and return a `pandas.Series` of the same shape and indexes as the original series object. However, instead of returning a series of `True`/`False` values, it performs some operation on the data that gets the column data closer to being how you want it to look during analysis operations.

## Writing Custom Recoders

### Recoders are Meant to be Composable

You are able to build a single, monolithic recoder that transforms a column all the way into what you want in a single step. But it may be better to write recoders that perform only a single step on the way to getting a particular column into shape. 

This results in reusable functions that are "composable". Meaning that for the NEXT column, you may not even need to write a new function. All you may need to do is list a sequence of recoders that already exist. 

This is because, recoders are applied in a pipeline model.  The output from the first is supplied to the second etc.

Now, lets write our recoders to transform `col4`.

In [14]:
def standardize_sex(series):
    """Return a series where common representations of 'male'/'female' are standardized.
    
    Things like ['M', 'MALE', 'M', 'BOY', ...] are converted to `M`.
    Representations of female are treated similarly. 
    """
    mapper = {"M": "M",
              "MALE": "M",
              "BOY": "M",
              "F": "F",
              "FEMALE": "F",
              "GIRL": "F",
             }
    if series.str.islower().any():
        raise ValueError("standardize_sex expects input series to contain only UPPERCASE letters.")
    else:
        return series.apply(lambda x: mapper[x])

Since we plan to call the built-in recoder `upper` first, we only need to support uppercase text here. This is kind of a silly requirement in practice but it demonstrates how you can write composable recoders to fail if expectations are not met.

## Next Iteration: Validate Recoded Table

Lets revisit our original definition of `col4` and add the recoders we have now.

In [15]:
col4 = Column(name='col4',
             dtype=str,
             unique=False,
             validators=[v.funcs.upper, length_is_one, valid_sex],
             recoders=[r.funcs.upper, standardize_sex])

Now what do we have after we recode this column?

In [16]:
col4.recode(df)

0    M
1    M
2    F
Name: col4, dtype: object

That looks pretty good. But let's not rely on our eyes and instead rely on the validation tests we defined. 

Now we try `col4.validate` again but this time tell it to perform recoding prior to validation and see what happens.

In [17]:
col4.validate(df, recode=True)

,length_is_one,upper,valid_sex,dtype
0,True,True,True,True
1,True,True,True,True
2,True,True,True,True


We are good! We can now move on to our other columns and repeat the process until all the validation check pass on all of our columns. Then we can compose our `Enforcer` object and we will be nearing the end of our initial sanity check / recoding phase for this table. 

# Defining Table Enforcers

In [26]:
demo = Enforcer(columns=[col1,col3,col4])

In [27]:
demo.validate(df)

False

In [28]:
demo.recode(df)

,col1,col3,col4
0,7,10,M
1,2,6,M
2,6,2,F


In [29]:
demo.validate(df, recode=True)

True

## Writing loading functions that accept enforcers

Pass enforcers to your loading functions to get table type specific recoding and validation for all of your table types in a single function.

In [22]:
def load_csv(path, enforcer):
    df = pd.read_csv(path)
    
    recoded = enforcer.recode(df)
    
    if enforcer.validate(recoded):
        return recoded
    else:
        raise e.ValidationError()
    

In [31]:
df

,col1,col2,col3,col4
0,7,3,10,male
1,2,4,6,m
2,6,6,2,F


In [30]:
df2 = load_csv(path="demo_table.csv", enforcer=demo)
df2

,col1,col3,col4
0,7,10,M
1,2,6,M
2,6,2,F
